In [ ]:
import os
import re
import sys
import math
import numpy as np
from numpy.linalg import inv
from scipy.signal import lsim
from scipy.fft import fft, fftfreq
from pathlib import Path
from tqdm.notebook import tqdm
if '..' not in sys.path:
    sys.path = ['..'] + sys.path
from filter_OU_inputs import fit_TF, filter_inputs
from multitone import newman_phase, compute_fourier_coeffs

In [ ]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
fontsize = 9
FIGURES_DIR = Path('figures')

In [ ]:
# data_dir = Path('../data/Sardinia/default_config/GFMs_01_02_03_noTG_lowLV/Ta_40')
data_dir = Path('../data/Sardinia/default_config/GFM_01_lowLV/Ta_2')
assert data_dir.exists(), f'{data_dir}: no such directory.'

Load the transient data: the data file contains the results of the simulations performed with PowerFactory.

In [ ]:
# N_tones, F0 = 2, 0.1
# fname = 'Sardegna_2021_06_03cr_3GFM_noTG_lowLV_tran_multitone_N_{}_F0_{:g}_Hz.npz'
N_tones, F0 = 32, 0.005
fname = 'Sardegna_2021_06_03cr_1GFM_lowLV_tran_multitone_N_{}_F0_{:g}_Hz.npz'
tran_data_file = data_dir / fname.format(N_tones, F0)
assert tran_data_file.exists(), f'{tran_data_file}: no such file'
blob = np.load(tran_data_file, allow_pickle=True)
tran_time = blob['time'].astype(float)
tran_data = blob['data'].item()
fe_bus = tran_data['bus']['m:fe']

In [ ]:
freqs = np.array([(i + 1) * F0 for i in range(N_tones)])
ttran = 1 / F0 / 2
idx = tran_time >= ttran
t = tran_time[idx] - ttran
Δω = fe_bus[idx, :] - 1
phase = newman_phase(np.arange(N_tones) + 1, N_tones)
coeffs = compute_fourier_coeffs(freqs, t, Δω, phase)

In [ ]:
k = 0
N_samples = Δω.shape[0]
yf = fft(Δω[:,k])
xf = fftfreq(N_samples, t[1] - t[0])[:N_samples//2]
yps = 2 / N_samples * np.abs(yf[:N_samples//2])

xlim = [freqs[0] / 2, freqs[-1] * 1.1]
fig, ax = plt.subplots(2, 1, figsize=(5,3.5))
ax[0].plot(t, Δω[:,k], 'k', lw=1)
ax[0].set_xlabel('Time [s]')
ax[0].set_ylabel(r'$X$')
idx = xf <= xlim[1]
ax[1].vlines(xf[idx], 0, yps[idx], color='k', lw=0.5, ls='-')
ax[1].vlines(freqs, 0, yps.max(), color='r', lw=1, ls=':')
ax[1].plot(xf, yps, 'ko', markerfacecolor='w', markersize=4)
ax[1].plot(freqs, np.abs(coeffs[:,0]), 's', color='tab:blue', markerfacecolor='w', markersize=4)
ax[1].set_xscale('log')
ax[1].set_xlim(xlim)
ax[1].set_xlabel('Frequency [Hz]')
ax[1].set_ylabel(r'$|\hat{X}(j\omega)|$')
sns.despine()
fig.tight_layout()